<h1 align="center">Simulations</h1>

<h2>Changing to correct directory</h2>

In [1]:
import os
from pathlib import Path

os.chdir(Path.cwd().parent)
assert Path.cwd().name == "src" # make sure it is the correct folder

In [2]:
import numpy as np
import pandas as pd

import tournament_simulations.data_structures as ds
import tournament_simulations.simulations as simul

<h2>Matches-Wide</h2>

> Each match has its own probability.

In [3]:
from example_notebooks.matches_df_example import matches_df_mw
matches_mw = ds.mat.Matches(matches_df_mw)
ppm_mw = ds.ppm.PointsPerMatch.from_home_away_winner(matches_mw.home_away_winner)

# 'probabilities' aren't a part of Matches, but we added here to keep everything together
matches_mw.df

home   away winner          probabilities
id  date number                                            
T/1 0             home   away      h              (1, 0, 0)
    0             home   away      a              (1, 0, 0)
    0             away   home      d              (0, 0, 1)
    0             home   away      a              (0, 0, 1)
    1             away   home      a              (0, 1, 0)
    1             home   away      h              (1, 0, 0)
T/2 0              one  three      a  (0.333, 0.333, 0.334)
    0              two    one      h  (0.333, 0.333, 0.334)
    1              two  three      a  (0.333, 0.333, 0.334)
    1              one    two      h  (0.333, 0.333, 0.334)
    2            three    two      h  (0.333, 0.333, 0.334)
    2            three    one      a  (0.333, 0.333, 0.334)

<h3>Winners</h3>

> Simulates winners
> - Home: "h" <br>
> - Draw: "d" <br>
> - Away: "a"

In [4]:
simul_matches_mw = simul.SimulateMatches(matches_mw)
simul_matches_mw.matches.df

home   away winner          probabilities
id  date number                                            
T/1 0             home   away      h              (1, 0, 0)
    0             home   away      a              (1, 0, 0)
    0             away   home      d              (0, 0, 1)
    0             home   away      a              (0, 0, 1)
    1             away   home      a              (0, 1, 0)
    1             home   away      h              (1, 0, 0)
T/2 0              one  three      a  (0.333, 0.333, 0.334)
    0              two    one      h  (0.333, 0.333, 0.334)
    1              two  three      a  (0.333, 0.333, 0.334)
    1              one    two      h  (0.333, 0.333, 0.334)
    2            three    two      h  (0.333, 0.333, 0.334)
    2            three    one      a  (0.333, 0.333, 0.334)

In [5]:
np.random.seed(1)

simul_matches_mw.match_wide(
    num_iteration_simulation=(3,3),
    match_to_probabilities=matches_mw.df["probabilities"],
    func_after_simulation=lambda x: x
)

s0 s1 s2 s3 s4 s5 s6 s7 s8
id  date number home  away                            
T/1 0           home  away   h  h  h  h  h  h  h  h  h
                      away   h  h  h  h  h  h  h  h  h
                away  home   a  a  a  a  a  a  a  a  a
                home  away   a  a  a  a  a  a  a  a  a
    1           away  home   d  d  d  d  d  d  d  d  d
                home  away   h  h  h  h  h  h  h  h  h
T/2 0           one   three  h  h  a  d  h  d  h  a  d
                two   one    a  h  a  h  d  a  d  d  h
    1           two   three  a  a  h  h  d  a  a  d  h
                one   two    h  h  a  d  h  d  d  h  d
    2           three two    h  d  a  d  d  a  a  d  a
                      one    d  a  h  d  a  h  d  h  a

In [6]:
np.random.seed(1)

# Function to be applied after each iteration
def count_number_of_homes_each_col(df: pd.DataFrame) -> pd.DataFrame:
    def number_of_homes(df: pd.DataFrame) -> pd.DataFrame:
        homes_per_col = lambda col: col.value_counts().to_dict().get("h", 0)
        return df.apply(homes_per_col)

    return df.groupby("id").apply(number_of_homes)

simul_matches_mw.match_wide(
    num_iteration_simulation=(3,3),
    match_to_probabilities=matches_mw.df["probabilities"],
    func_after_simulation=count_number_of_homes_each_col,
)

,s0,s1,s2,s3,s4,s5,s6,s7,s8
id,,,,,,,,,
T/1,3,3,3,3,3,3,3,3,3
T/2,3,3,2,2,2,1,1,2,2


<h3>Points Per Match</h3>

> Simulates points each team made in each match they played
> - Win:  3 <br>
> - Draw: 1 <br>
> - Loss: 0

In [7]:
simul_ppm_mw = simul.SimulatePointsPerMatch(ppm_mw)
simul_ppm_mw.ppm.df

team  points
id  date number               
T/1 0             home       3
    0             away       0
    0             home       0
    0             away       3
    0             away       1
    0             home       1
    0             home       0
    0             away       3
    1             away       0
    1             home       3
    1             home       3
    1             away       0
T/2 0              one       0
    0            three       3
    0              two       3
    0              one       0
    1              two       0
    1            three       3
    1              one       3
    1              two       0
    2            three       3
    2              two       0
    2            three       0
    2              one       3

In [8]:
np.random.seed(1)

simul_ppm_mw.match_wide(
    num_iteration_simulation=(3, 3),
    match_to_probabilities=matches_mw.df["probabilities"],
    func_after_simulation=lambda x: x,
)

s0  s1  s2  s3  s4  s5  s6  s7  s8
id  date number team                                     
T/1 0           home    3   3   3   3   3   3   3   3   3
                away    0   0   0   0   0   0   0   0   0
                home    3   3   3   3   3   3   3   3   3
                away    0   0   0   0   0   0   0   0   0
                away    0   0   0   0   0   0   0   0   0
                home    3   3   3   3   3   3   3   3   3
                home    0   0   0   0   0   0   0   0   0
                away    3   3   3   3   3   3   3   3   3
    1           away    1   1   1   1   1   1   1   1   1
                home    1   1   1   1   1   1   1   1   1
                home    3   3   3   3   3   3   3   3   3
                away    0   0   0   0   0   0   0   0   0
T/2 0           one     3   3   0   1   3   1   3   0   1
                three   0   0   3   1   0   1   0   3   1
                two     0   3   0   3   1   0   1   1   3
                one     3   0   3   0   1   3   1   1   0
    1           two     0   0   3   3   1   0   0   1   3
                three   3   3   0   0   1   3   3   1   0
                one     3   3   0   1   3   1   1   3   1
                two     0   0   3   1   0   1   1   0   1
    2           three   3   1   0   1   1   0   0   1   0
                two     0   1   3   1   1   3   3   1   3
                three   1   0   3   1   0   3   1   3   0
                one     1   3   0   1   3   0   1   0   3

In [9]:
np.random.seed(1)

# Function to be applied after each iteration
def rankings(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby(["id", "team"], observed=True).sum()

simul_ppm_mw.match_wide(
    num_iteration_simulation=(3, 3),
    match_to_probabilities=matches_mw.df["probabilities"],
    func_after_simulation=rankings,
)

s0  s1  s2  s3  s4  s5  s6  s7  s8
id  team                                     
T/1 away    4   4   4   4   4   4   4   4   4
    home   13  13  13  13  13  13  13  13  13
T/2 one    10   9   3   3  10   5   6   4   5
    three   7   4   6   3   2   7   4   8   1
    two     0   4   9   8   3   4   5   3  10

<h2>Tournament-Wide</h2>

> Each tournament has its own probability.

In [10]:
from example_notebooks.matches_df_example import matches_df_tw
matches_tw = ds.mat.Matches(matches_df_tw.sort_index())
ppm_tw = ds.ppm.PointsPerMatch.from_home_away_winner(matches_tw.home_away_winner)

matches_tw.probabilities_per_id  # estimated map

id
always_away_win                                      (0.0, 0.0, 1.0)
always_draw                                          (0.0, 1.0, 0.0)
always_home_win                                      (1.0, 0.0, 0.0)
uniform_prob       (0.3333333333333333, 0.3333333333333333, 0.333...
Name: probabilities, dtype: object

<h3>Winners</h3>

> Simulates winners
> - Home: "h" <br>
> - Draw: "d" <br>
> - Away: "a"

In [11]:
simul_matches_tw = simul.SimulateMatches(matches_tw)
simul_matches_tw.matches.df

home   away winner
id              date number                     
always_away_win 0             home   away      a
                1             home   away      a
always_draw     0             home   away      d
                0             home   away      d
always_home_win 0             home   away      h
                1             home   away      h
uniform_prob    0              one  three      d
                0              two    one      d
                1              two  three      a
                1              one    two      h
                2            three    two      h
                2            three    one      a

In [12]:
np.random.seed(1)

simul_matches_tw.tournament_wide(
    num_iteration_simulation=(3, 3),
    id_to_probabilities=matches_tw.probabilities_per_id,  # provinding a map
    func_after_simulation=lambda x: x,
)

s0 s1 s2 s3 s4 s5 s6 s7 s8
id              date number home  away                            
always_away_win 0           home  away   a  a  a  a  a  a  a  a  a
                1           home  away   a  a  a  a  a  a  a  a  a
always_draw     0           home  away   d  d  d  d  d  d  d  d  d
                                  away   d  d  d  d  d  d  d  d  d
always_home_win 0           home  away   h  h  h  h  h  h  h  h  h
                1           home  away   h  h  h  h  h  h  h  h  h
uniform_prob    0           one   three  h  h  a  d  h  d  h  a  d
                            two   one    a  h  a  h  d  a  d  d  h
                1           two   three  a  a  h  h  d  a  a  d  h
                            one   two    h  h  a  d  h  d  d  h  d
                2           three two    h  d  a  d  d  a  a  d  a
                                  one    d  a  h  d  a  h  d  h  a

In [13]:
np.random.seed(1)

simul_matches_tw.tournament_wide(
    num_iteration_simulation=(3, 3),
    id_to_probabilities=None,
    func_after_simulation=count_number_of_homes_each_col,
)

,s0,s1,s2,s3,s4,s5,s6,s7,s8
id,,,,,,,,,
always_away_win,0,0,0,0,0,0,0,0,0
always_draw,0,0,0,0,0,0,0,0,0
always_home_win,2,2,2,2,2,2,2,2,2
uniform_prob,3,3,2,2,2,1,1,2,2


<h3>Points Per Match</h3>

> Simulates points each team made in each match they played
> - Win:  3 <br>
> - Draw: 1 <br>
> - Loss: 0

In [14]:
simul_ppm_tw = simul.SimulatePointsPerMatch(ppm_tw)
simul_ppm_tw.ppm.df

team  points
id              date number               
always_away_win 0             home       0
                0             away       3
                1             home       0
                1             away       3
always_draw     0             home       1
                0             away       1
                0             home       1
                0             away       1
always_home_win 0             home       3
                0             away       0
                1             home       3
                1             away       0
uniform_prob    0              one       1
                0            three       1
                0              two       1
                0              one       1
                1              two       0
                1            three       3
                1              one       3
                1              two       0
                2            three       3
                2              two       0
                2            three       0
                2              one       3

In [15]:
np.random.seed(1)

simul_ppm_tw.tournament_wide(
    num_iteration_simulation=(3, 3),
    id_to_probabilities=matches_tw.probabilities_per_id,
    func_after_simulation=lambda x: x,
)

s0  s1  s2  s3  s4  s5  s6  s7  s8
id              date number team                                     
always_away_win 0           home    0   0   0   0   0   0   0   0   0
                            away    3   3   3   3   3   3   3   3   3
                1           home    0   0   0   0   0   0   0   0   0
                            away    3   3   3   3   3   3   3   3   3
always_draw     0           home    1   1   1   1   1   1   1   1   1
                            away    1   1   1   1   1   1   1   1   1
                            home    1   1   1   1   1   1   1   1   1
                            away    1   1   1   1   1   1   1   1   1
always_home_win 0           home    3   3   3   3   3   3   3   3   3
                            away    0   0   0   0   0   0   0   0   0
                1           home    3   3   3   3   3   3   3   3   3
                            away    0   0   0   0   0   0   0   0   0
uniform_prob    0           one     3   3   0   1   3   1   3   0   1
                            three   0   0   3   1   0   1   0   3   1
                            two     0   3   0   3   1   0   1   1   3
                            one     3   0   3   0   1   3   1   1   0
                1           two     0   0   3   3   1   0   0   1   3
                            three   3   3   0   0   1   3   3   1   0
                            one     3   3   0   1   3   1   1   3   1
                            two     0   0   3   1   0   1   1   0   1
                2           three   3   1   0   1   1   0   0   1   0
                            two     0   1   3   1   1   3   3   1   3
                            three   1   0   3   1   0   3   1   3   0
                            one     1   3   0   1   3   0   1   0   3

In [16]:
np.random.seed(1)

simul_ppm_tw.tournament_wide(
    num_iteration_simulation=(3, 3),
    id_to_probabilities=matches_tw.probabilities_per_id,
    func_after_simulation=rankings,
)

s0  s1  s2  s3  s4  s5  s6  s7  s8
id              team                                     
always_away_win away    6   6   6   6   6   6   6   6   6
                home    0   0   0   0   0   0   0   0   0
always_draw     away    2   2   2   2   2   2   2   2   2
                home    2   2   2   2   2   2   2   2   2
always_home_win away    0   0   0   0   0   0   0   0   0
                home    6   6   6   6   6   6   6   6   6
uniform_prob    one    10   9   3   3  10   5   6   4   5
                three   7   4   6   3   2   7   4   8   1
                two     0   4   9   8   3   4   5   3  10